<a href="https://colab.research.google.com/github/nbhushan/ColabSandbox/blob/main/XAI_DataLab1_PDP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Applying Partial Dependence Plot (PDP) to understand what drives bike rentals.

In this notebook, we will explore the use of PDPs to understand the factors that drive bike rentals. You are free to use any predictive model of your choice.

1. Load the required libraries

In [ ]:
# base packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# ML packages
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn import metrics
from sklearn.metrics import r2_score

#XAI packages
from sklearn.inspection import PartialDependenceDisplay

2. Download the BikeRentalsDataset from the UCI repository.

This dataset contains daily counts of rented bicycles from the bicycle rental company Capital-Bikeshare in Washington D.C., along with weather and seasonal information. The goal is to predict how many bikes will be rented depending on the weather and the day. 

In [ ]:
# Since we are using colab, we can directly use linux commands such as wget, unzip and ls
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00275/Bike-Sharing-Dataset.zip 
!unzip Bike-Sharing-Dataset.zip 
!ls

3. Exploratory Data Analysis

In [ ]:
# use the data catalog to understand the features (https://christophm.github.io/interpretable-ml-book/bike-data.html)
df = pd.read_csv("day.csv")
df.head()

Next, let's check the data types of the features.



In [ ]:
#keep only relevant columns (https://christophm.github.io/interpretable-ml-book/bike-data.html)

df.drop(columns=['instant', 'dteday', 'registered', 'casual', 'yr'], axis=0, inplace=True)


Clearly, we need to convert some features to categorical before we do the analysis.

In [ ]:
# use a for-loop to convert the categorical features to the categorical data type and create dummy variables

categorical_columns = ['season', 'holiday', 'weekday', 'workingday', "weathersit", "mnth"]
for col in categorical_columns:
    df[col] = df[col].astype('category')

# verify that the change is now applied
df.dtypes

In [ ]:
#create dummy variables (one hot encoding)

df_onehot = pd.get_dummies(df, columns = categorical_columns, drop_first=True)

df_onehot.head()

In [ ]:
#Verify data types

df_onehot.dtypes

Let's make some plots to understand the data.

In [ ]:
plt.figure(figsize=(15, 5))
# Using subplots to generate multiple plots side-by-side
plt.subplot(131)
plt.scatter('temp', 'cnt', data=df_onehot)
plt.xlabel('temperature')
plt.ylabel('bike rentals')
plt.subplot(132)
plt.scatter('windspeed', 'cnt', data=df_onehot)
plt.xlabel('windspeed')
plt.subplot(133)
plt.scatter('hum', 'cnt', data=df_onehot)
plt.xlabel('humidity')
plt.suptitle('Key Features predicting bike rentals')
plt.show()


In [ ]:
#view correlation matrix

df_onehot.corr()

## Add your observations here

Comment on the nature of the relationship between the three features and the 
outcome

(hint: which variable would you remove from your analysis?)

# Predictive Model

First, we will divide the data into features and outcome. Our outcome is what we are trying to predict (the count of bike rentals on a given day), and our features are our input into that prediction.

For this example I will focus on the following variables as features.

['temp', 'hum', 'windspeed', 'weekday', 'workingday', 'holiday'] 



In [ ]:
#Create feature vector
features = df_onehot.loc[:, df_onehot.columns != 'cnt']
#Create outcome vector
outcome = df_onehot['cnt']

Split the data into traiing and test sets


In [ ]:
# Split data into train and test set: 70% / 30%
X_train, X_test, y_train, y_test = train_test_split(features, outcome, test_size=0.3, random_state=0)


Let's build a decision tree using default parameters, train the model and generate predictions.

In [ ]:
#build the model
mod_DT = DecisionTreeRegressor()
#model fit
mod_DT.fit(X_train, y_train)
# make predictions
y_pred = mod_DT.predict(X_test)

## Model Interpretation

Let's use PDPs to understand how the model makes it's predictions.

In [ ]:
pdp_features = ['temp', 'hum', 'windspeed']

fig, ax = plt.subplots(figsize=(12, 6))
ax.set_title("PDP: Decision tree regression")
pdp_RF = PartialDependenceDisplay.from_estimator(mod_DT, X_train, pdp_features, 
                                                 ax=ax, grid_resolution=5)

The left plot shows the partial dependence between bike rentals and temperature.

We notice that PDPs are always generated after we fit the model, in this case we fit the model using temperature, humidity, windspeed as features.


We then now use the model to predict rentals on a given day D with a set humidity and windspeed, but only vary the temperature. We first predict the number of rentals for the day when the temperature is 0.2 (recall that temperature is normalized). We then repeat it for 0.4, 0.6..and so on until we reach the maximum range that temeprature can reach (1 because of normalization). In this way we trace out how the bike rentals change as we vary temeprature, keeping humidity and windspeed fixed (or "integrating" out humidity and windspeed).

In reality, what you observe is the plot obtained by performing the same guessing game for several days and averaging them to get a robust idea of how the model is able to make it's predictions.

In the left graph, we see rentals tend to increase and temperatures increase, but at a certain point, they start to dip because the temperatures start to reach their maximum values. In other words, it's too hot to bike.

The right graph shows the impact of windspeed and humidity which have clearer (almost linear) interpretations.

## Model Evaluation

To evaluate how well our model performed, we'll look at three metrics:


*   MAE: the average of all absolute errors.
*   MSE: takes the distance of all points from the regression line (residuals) and returns the squared average.
*   RMSE: the standard deviation of the residuals.





In [ ]:
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

We can determine how well the regression model fits the observed data with the r-squared value. R-squared is a value between 0 and 1, with a higher r-squred value generally indicating a better fit. We will round to two decimal places.

In [ ]:
print('R_squared: %.2f'
      % r2_score(y_test, y_pred))

Our r-squared value is X, which means that our model explains X% of the differences between the actual rental count and the predicted count. 

# Challenge

Build a predictive model that can improve on the R_squared of X and interpret
the model using PDPs